# Answers - OC Lab 2


## 2.1


In [14]:
from typing import List
from tabulate import tabulate

table_rows: List[List[str | float]] = [
    ["Array Size", "Avg Elapsed Time (s)", "Number of accesses", "Avg Access Time (ns)"]
]

with open("lab2_kit/spark/spark.log") as f:
    next(f)  # ignore header

    array_size = None
    elapsed_times = []
    access_times = []

    def close_block():
        global array_size, elapsed_times, access_times, num_accesses

        if array_size is None:
            return

        avg_elapsed_time = sum(elapsed_times) / len(elapsed_times)
        avg_access_time = sum(access_times) / len(access_times)

        table_rows.append([array_size, avg_elapsed_time, num_accesses, avg_access_time])

        array_size = None
        elapsed_times = []
        access_times = []

    for line in f:
        if line.startswith("[LOG]:"):
            close_block()
            array_size = line[line.find("size ") + len("size ") :]
        elif array_size is not None:
            _, _, elapsed, _, access, num_accesses = line.split("\t")
            elapsed_times.append(float(elapsed))
            access_times.append(float(access))
        else:
            print("Error: malformed logfile")
            exit(1)

    close_block()

tabulate(table_rows, headers="firstrow", tablefmt="html")


Array Size,Avg Elapsed Time (s),Number of accesses,Avg Access Time (ns)
4 KiB,0.00133117,409600,3.24993
8 KiB,0.00195577,819200,2.38746
16 KiB,0.00391229,1638400,2.38788
32 KiB,0.00790633,3276800,2.4128
64 KiB,0.0190731,6553600,2.91031
128 KiB,0.0417712,13107200,3.18689
256 KiB,0.0910264,26214400,3.47238
512 KiB,0.195436,52428800,3.72765
1024 KiB,0.407893,104857600,3.88997
2048 KiB,0.85298,209715200,4.06732


| **Array Size**             | 8 KiB          | 16 KiB         | 32 KiB        | 64 KiB         | 128 KiB        | 256 KiB        |
| -------------------------- | -------------- | -------------- | ------------- | -------------- | -------------- | -------------- |
| **t2-t1 (s)**              | 0.00195577     | 0.00391229     | 0.00790633    | 0.0190731      | 0.0417712      | 0.0910264      |
| **# accesses a[i]**        | 819200         | 1638400        | 3276800       | 6553600        | 13107200       | 26214400       |
| **# mean access time (s)** | 2.38746 × 10⁻⁹ | 2.38788 × 10⁻⁹ | 2.4128 × 10⁻⁹ | 2.91031 × 10⁻⁹ | 3.18689 × 10⁻⁹ | 3.47238 × 10⁻⁹ |


By analyzing the table above, representative of data gathered by running the Spark program on the lab7p2 computer, we can empirically assess that the machine's cache capacity is 32KB. We can conclude this because the mean access time clearly increases for array sizes above 32KB (from around 2.4128ns to 2.91031ns and above).

![plot](assets/2.1.png)


## 2.2


The cache size is 64KB because from there the reading and writing times increase in a disproportional manner - there is a clear spike between 64KB and 128KB for the same steps in stride, due to an increase in capacity misses.


## 2.3


By analyzing the plots for array sizes that do not fully fit in the cache (>=128KB), we can see that reading and writing times stabilize after a stride size of 16, thus we can conclude that each cache block has 16B of size.


## 2.4


For a stride of 16 (per 2.3), the maximum array size that fits in the cache (64KB per 2.2) has an average reading and writing time of approximately 375ns, while for array sizes that do not fit in the cache this time is consistently around 975ns. As such, we can conclude that the L1 miss penalty time is approximately $975 - 375 = 600ns$.


## 3.1


### 3.1.1


#### a)


During the program's execution, the analyzed events will be L1 Data Cache misses (PAPI_L1_DCM) - these will be triggered every time data is not found in the L1 cache and it is necessary to access the next level of the memory hierarchy (L2 cache or, if there isn't one, main memory).


#### b)


Tabela: está em cm1.out, depois vamos lá buscar os valores :fixe3:


![Plot](assets/3.1.1-b.png)


#### c)


##### L1 size:


Above 32KB, the average miss rate spikes disproportionally. We can therefore assume that the whole array fit in cache previously (not fitting anymore for 64KB), and that as the array size surpasses that value, the misses start to become overwhelming. As such, we can conclude that the L1 cache size is 32KB.


##### Block size:


The cache block size is 64B, as strides up to 64 show an increasing miss rate, stabilizing at 1 after reaching 64 up to a stride of 1024. As strides represent how many `uint8_t`s we skip and `uint8_t`s take up 1 Byte each, we can infer that we are essentially always loading a new block into cache with a stride of 64 (as the word in question is guaranteed not to be in cache). It can also be noted that for strides of 8, 16 and 32 words, the miss rate also grows from 12.5 to 25 to 50%, effectively doubling the miss rate for each stride increase - e.g., if we jump in groups of 8 words with a 64B block size, we're bound to have to load a new block every 8 times, and so on.


##### Associativity set size:


Since the cache size is 32KB, the index + offset take up 15 bits of the address. Considering the 64KB array (which doesn't fit completely in the cache), for a stride of $2^{15}$, the same 2 items of the array are being accessed continuously and mapped to the same index. This means that if there was no associativity, a >=100% miss-rate would be expected. However that is not the case (the miss-rate is actually 0%) which can only mean associativity is >= 2. The same applies to strides of $2^{14}$ and $2^{13}$. Since this behavior stops at strides of $2^{12}$ (miss-rate jumps to >=100%), we can then conclude the associativity set size is 8.


### 3.1.2


#### a)


We changed both the event being tracked to `PAPI_L2_DCM`, to be able to track L2 Data Cache misses now, and the `CACHE_MIN` and `CACHE_MAX` values, respectively to 64KiB and 1MiB, to be able to track the L2 cache. Do note that the 64KiB value was explicitly chosen as to start exactly one power of 2 above the expected L1 cache size, since the L2 cache is supposed to always be bigger than the L1 cache.


#### b)


![Plot](assets/3.1.2-b.png)


#### c)


##### L2 size:


For the same reasons described in 3.1.1.c)'s L1 cache size section, the L2 cache size seems to be 256KiB, as if array size goes above that, the miss rate spikes to 1: the array doesn't fit in cache as a whole anymore, which leads to misses starting to happen.


##### Block size:


For the same reasons described in 3.1.1.c)'s Block size section, the L2 cache's block size also seems to be 64B, as after a stride size of 64 the miss rate stabilizes at 1.


##### Associativity set size:


As explained in 3.1.1.c)'s Associativity set size section, the L2 set size seems to be 32, as the miss-rate jumps from 0% to 50% from strides of $2^{15}$ to strides of $2^{14}$, given an array size of 1MB, for example.


## 3.2


### 3.2.1


#### a)


We have two $512 \times 512$ `uint16_t` matrices: as each `unit16_t` occupies 2B, each matrix occupies $512^2 \times 2 = \left(2^9\right)^2 \times 2 = 2^{19}$ Bytes = $512$ KB, so the two of them combined occupy $2 \times 2^{19} = 2^{20}$ Bytes, or $1$ MB, in memory.


#### b)


Program output:

```
After resetting counter 'PAPI_L1_DCM' [x10^6]: 0.000000
After resetting counter 'PAPI_LD_INS' [x10^6]: 0.000000
After resetting counter 'PAPI_SR_INS' [x10^6]: 0.000000
After stopping counter 'PAPI_L1_DCM'  [x10^6]: 134.444855
After stopping counter 'PAPI_LD_INS'  [x10^6]: 3491.023749
After stopping counter 'PAPI_SR_INS'  [x10^6]: 672.141375
Wall clock cycles [x10^6]: 3995.673182
Wall clock time [seconds]: 1.177878
Matrix checksum: 2717908992
```


| **Total number of L1 data cache misses**                | 134.444855                             | **⨯ 10^6**  |
| ------------------------------------------------------- | -------------------------------------- | ----------- |
| **Total number of load / store instructions completed** | 3491.023749 + 672.141375 = 4163.165124 | **⨯ 10^6**  |
| **Total number of clock cycles**                        | 3995.673182                            | **⨯ 10^6**  |
| **Elapsed time**                                        | 1.177878                               | **seconds** |


#### c)


$$
\operatorname{HitRate} = 1 - \operatorname{MissRate} = 1 - \frac{\operatorname{Misses}}{\operatorname{Accesses}} = 1 - \frac{134.444855 \times 10^6}{4163.165124 \times 10^6} \approx 0.9677
$$


### 3.2.2


#### a)


Program output:

```
After resetting counter 'PAPI_L1_DCM' [x10^6]: 0.000000
After resetting counter 'PAPI_LD_INS' [x10^6]: 0.000000
After resetting counter 'PAPI_SR_INS' [x10^6]: 0.000000
After stopping counter 'PAPI_L1_DCM'  [x10^6]: 4.212926
After stopping counter 'PAPI_LD_INS'  [x10^6]: 402.664929
After stopping counter 'PAPI_SR_INS'  [x10^6]: 134.217780
Wall clock cycles [x10^6]: 744.145336
Wall clock time [seconds]: 0.219365
Matrix checksum: 2717908992
```


| **Total number of L1 data cache misses**                | 4.212926                             | **⨯ 10^6**  |
| ------------------------------------------------------- | ------------------------------------ | ----------- |
| **Total number of load / store instructions completed** | 402.664929 + 134.217780 = 536.882709 | **⨯ 10^6**  |
| **Total number of clock cycles**                        | 744.145336                           | **⨯ 10^6**  |
| **Elapsed time**                                        | 0.219365                             | **seconds** |


#### b)


$$
\operatorname{HitRate} = 1 - \operatorname{MissRate} = 1 - \frac{\operatorname{Misses}}{\operatorname{Accesses}} = 1 - \frac{4.212926 \times 10^6}{536.882709 \times 10^6} \approx 0.9922
$$


#### c)


Program output:

```
After resetting counter 'PAPI_L1_DCM' [x10^6]: 0.000000
After resetting counter 'PAPI_LD_INS' [x10^6]: 0.000000
After resetting counter 'PAPI_SR_INS' [x10^6]: 0.000000
After stopping counter 'PAPI_L1_DCM'  [x10^6]: 4.484165
After stopping counter 'PAPI_LD_INS'  [x10^6]: 402.925461
After stopping counter 'PAPI_SR_INS'  [x10^6]: 134.479925
Wall clock cycles [x10^6]: 744.901308
Wall clock time [seconds]: 0.219588
Matrix checksum: 2717908992
```


| **Total number of L1 data cache misses**                | 4.484165                            | **⨯ 10^6**  |
| ------------------------------------------------------- | ----------------------------------- | ----------- |
| **Total number of load / store instructions completed** | 402.925461 + 134.479925 = 537.40586 | **⨯ 10^6**  |
| **Total number of clock cycles**                        | 744.901308                          | **⨯ 10^6**  |
| **Elapsed time**                                        | 0.219588                            | **seconds** |


$$
\operatorname{HitRate} = 1 - \operatorname{MissRate} = 1 - \frac{\operatorname{Misses}}{\operatorname{Accesses}} = 1 - \frac{4.484165 \times 10^6}{537.40586 \times 10^6} \approx 0.9917
$$


Even though including the transposition increased all the values in question, the difference is negligible as the time complexity of transposition (quadratic) is much smaller than the one associated with matrix multiplication (cubic).


#### d)


| **$\Delta\text{HitRate} = \text{HitRate}_{\text{mm2}} - \text{HitRate}_{\text{mm1}}$**         | $0.9917 - 0.9677 = 0.0240$                     |
| ---------------------------------------------------------------------------------------------- | ---------------------------------------------- |
| **$\text{Speedup(\#Clocks)} = \frac{\text{Clocks}_{\text{mm1}}}{\text{Clocks}_{\text{mm2}}}$** | $\frac{3995.673182}{744.901308} = 5.364030294$ |
| **$\text{Speedup(Time)} = \frac{\text{Time}_{\text{mm1}}}{\text{Time}_{\text{mm2}}}$**         | $\frac{1.177878}{0.219588} = 5.364036286$      |


The speedup gained by this second implementation seems to be truly worthwhile, showing a considerable improvement in terms of the hit rate, clock cycles and time gained.


### 3.2.3


#### a)


Each element occupies $2$ Bytes, since they are of type `uint16_t`. Therefore, since each cache line takes $64$ bytes (per Section 3.1), the number of elements per line is $\frac{64}{2} = 32$.


#### b)


Program output:

```
After resetting counter 'PAPI_L1_DCM' [x10^6]: 0.000000
After resetting counter 'PAPI_LD_INS' [x10^6]: 0.000000
After resetting counter 'PAPI_SR_INS' [x10^6]: 0.000000
After stopping counter 'PAPI_L1_DCM'  [x10^6]: 5.405940
After stopping counter 'PAPI_LD_INS'  [x10^6]: 402.659833
After stopping counter 'PAPI_SR_INS'  [x10^6]: 134.217797
Wall clock cycles [x10^6]: 395.745072
Wall clock time [seconds]: 0.116661
Matrix checksum: 2717908992
```


| **Total number of L1 data cache misses**                | 5.405940                            | **⨯ 10^6**  |
| ------------------------------------------------------- | ----------------------------------- | ----------- |
| **Total number of load / store instructions completed** | 402.659833 + 134.217797 = 536.87763 | **⨯ 10^6**  |
| **Total number of clock cycles**                        | 395.745072                          | **⨯ 10^6**  |
| **Elapsed time**                                        | 0.116661                            | **seconds** |


#### c)


$$
\operatorname{HitRate} = 1 - \operatorname{MissRate} = 1 - \frac{\operatorname{Misses}}{\operatorname{Accesses}} = 1 - \frac{5.405950 \times 10^6}{536.87763 \times 10^6} \approx 0.9899
$$


#### d)


| **$\Delta\text{HitRate} = \text{HitRate}_{\text{mm2}} - \text{HitRate}_{\text{mm1}}$**         | $0.99 - 0.9677 = 0.0223$                     |
| ---------------------------------------------------------------------------------------------- | ---------------------------------------------- |
| **$\text{Speedup(\#Clocks)} = \frac{\text{Clocks}_{\text{mm1}}}{\text{Clocks}_{\text{mm3}}}$** | $\frac{3995.673182}{395.745072} = 10.0965835451$ |


This new implementation, exploring the spacial locality of the matrix, is much more efficient than the original, "naive" one, leading to a speedup of around $10$ times.


#### e)


| **$\Delta\text{HitRate} = \text{HitRate}_{\text{mm3}} - \text{HitRate}_{\text{mm2}}$**         | 0.99 - 0.9917 = -0.0017                     |
| ---------------------------------------------------------------------------------------------- | --------------------------------------------- |
| **$\text{Speedup(\#Clocks)} = \frac{\text{Clocks}_{\text{mm2}}}{\text{Clocks}_{\text{mm3}}}$** | $\frac{744.901308}{395.745072} = 1.88227563829$ |


This new implementation ends up being even better than the transposition one, by a factor of about 1.88 times - it shows that, here, exploring the cache's spacial locality is more efficient than relying on a supposed algorithmic advantage, which might not even work well for very large matrixes.

We also, just as the question's statement suggested, went and tested the L2 miss events for both programs:

-   for the transposition one, we found 4.473961 L2 misses;
-   for the last one, we found 0.471593 L2 misses.

This is crucial for the program's efficiency, since going up a level in the cache, be it L3 or memory, is very costly (in comparison with L1 and L2), so it makes up for the overall hit rate being slightly lower.


### 3.2.3


After running the `lscpu` utility on the `lap7p2` machine (and seeing the CPU's specification - an Intel i5-3570 CPU), we asserted that the L1's size is of 2\*128KB for the whole processor - one data cache and one instruction cache for each of the four cores, therefore making it a 32KB cache per-core, per type of cache, just as expected.

Moreover, the L2's expected size also matches the experimentally attained one - `lscpu` returns an L2 cache size of 1MB, so 256KB per core, just as expected.

L1 has an associativity of 8 ways, as expected; however, L2 associativity is also 8, instead of the expected 32 ways. This is probably due to memory optimization mechanisms that emulate a higher associativity.